# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
###Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [62]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
%cd "/content/drive/My Drive/Data Job/Colab Notebooks/Data Science Nanodegree/Exercise Files/udacity-nanodegree-exercise/4_Data_Engineering/Lesson_5_Project"


/content/drive/My Drive/Data Job/Colab Notebooks/Data Science Nanodegree/Exercise Files/udacity-nanodegree-exercise/4_Data_Engineering/Lesson_5_Project


###Imports

In [47]:
# import libraries
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd
import math
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib import rcParams
from wordcloud import WordCloud, STOPWORDS

from sklearn.multioutput import MultiOutputClassifier
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
nltk.download(['punkt', 'wordnet', 'stopwords'])

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import f1_score
from sklearn.metrics import  confusion_matrix, classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


###Load Data

In [64]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql("SELECT * FROM disaster_resp_mes;", engine)
df.head()

OperationalError: ignored

In [58]:

inspector = inspect(engine)
schema = inspector.get_schema_names()[0]
colnames = []
table_name = inspector.get_table_names(schema=schema)[0]
for column in inspector.get_columns(table_name, schema=schema):
    colnames.append(column['name'])
target_colnames = colnames[4:]


IndexError: ignored

In [50]:
X = df['message']
Y = df[target_colnames]

###Functions

In [9]:
def tokenize(text):
    """
    INPUT:
    text - string
    OUTPUT:
    tokens - list of strings
    
    function takes raw text, removes punctuation signs, substitutes
    with spaces. Puts all characters in lower case, tokenizes text
    by words, removes stop words, lemmatizes, and returns list of tokens 
    """
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [14]:
def prep_wordcloud(df, colnames):
  """
  Function that modifies NLP dataset into lists that are ready to create wordcloud from

  INPUT:
    df - dataframe
    colnames - list of target column names 
  OUTPUT
    list of string, where each element is a string from each target
  """
  wordcloud_list = []
  for colum in colnames:

    #tokenize messages of each target
    wordcloud_set = df[df[colum]==1]['message']
    wordcloud_plot = ' '.join(wordcloud_set)
    wordcloud_plot = tokenize(wordcloud_plot)
    wordcloud_plot = ' '.join(wordcloud_plot)

    
    wordcloud_list.append(wordcloud_plot)

  return wordcloud_list

In [23]:
def display_results(y_test, y_pred, grid = None):
  f1_scores = []
  for ind, cat in enumerate(y_test):
    print('Class - {}'.format(cat))
    print(classification_report(y_test.values[ind], y_pred[ind], zero_division = 1))
    #display_results(y_test.values[ind], y_pred[ind])
    f1_scores.append(f1_score(y_test.values[ind], y_pred[ind]))
  
  print('Base Model\nMinimum f1 score - {}\nBest f1 score - {}\nMean f1 score - {}'.format(min(f1_scores), max(f1_scores), round(sum(f1_scores)/len(f1_scores), 3)))
  if grid:
    print("\nBest Parameters:", cv.best_params_)

###Feature Research, Visualizations
Search for messages that contain the word "update", plot the number of times this word is used in the message to the number of times it relates to any of target variables

In [51]:
df['genre'].unique()

array(['direct', 'social', 'news'], dtype=object)

In [52]:
df_cat = pd.get_dummies(df['genre'], dummy_na = False, drop_first=True)
df_cat.head()

,news,social
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [53]:
genre_mes = df.groupby(by = 'genre').sum()

In [54]:
fig = make_subplots(rows=6, cols=6, subplot_titles=target_colnames)

for i, nam in enumerate(target_colnames):
  fig.add_trace(go.Bar(x=genre_mes[nam].index, y=genre_mes[nam].values, name=nam),
                row=math.ceil((i+1) / 6), col= (i % 6) + 1)
  
fig.update(layout_showlegend=False)
fig.update_layout(height=1200, width=1200, title_text="Genre of messages per target")
fig.show()

It looks like the messages that come from 'social' are rarely classified into any of categories, 'direct' and 'news' are most sources of important information 

In [44]:
df[df['child_alone']==1]

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report


The entire dataset

###WordCloud

In [ ]:
word_list = prep_wordcloud(df, target_colnames)

In [ ]:
stopwords = STOPWORDS
#on initial run, the word 'people' was present in almost all categories
stopwords.add('people')

for i, cloud in enumerate(word_list):
  if len(cloud):
    print('Word Cloud for {} category'.format(target_colnames[i]))
    wordcloud = WordCloud(stopwords=stopwords, background_color="white",
                          max_words=20).generate(cloud)
    rcParams['figure.figsize'] = 10, 30
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

###Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [15]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultiOutputClassifier(RandomForestClassifier()))]) 

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)


In [16]:

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [24]:
display_results(y_test, y_pred)


Class - related
              precision    recall  f1-score   support

           0       1.00      0.97      0.98        31
           1       0.83      1.00      0.91         5

    accuracy                           0.97        36
   macro avg       0.92      0.98      0.95        36
weighted avg       0.98      0.97      0.97        36

Class - request
              precision    recall  f1-score   support

           0       0.69      1.00      0.81        24
           1       1.00      0.08      0.15        12

    accuracy                           0.69        36
   macro avg       0.84      0.54      0.48        36
weighted avg       0.79      0.69      0.59        36

Class - offer
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        31
           1       1.00      0.60      0.75         5

    accuracy                           0.94        36
   macro avg       0.97      0.80      0.86        36
weighted avg       0.95     

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.



              precision    recall  f1-score   support

           0       1.00      1.00      1.00        35
           1       1.00      1.00      1.00         1

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36

Class - storm
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        35
           1       1.00      1.00      1.00         1

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36

Class - fire
              precision    recall  f1-score   support

           0       1.00      0.97      0.99        36
           1       0.00      1.00      0.00         0

    accuracy                           0.97        36
   macro avg       0.50      0.99      0.49        36
weighted avg       1.00      0.97      0.99    

### 6. Improve your model
Use grid search to find better parameters. 

In [25]:
pipeline.get_params()

{'clf': MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0

In [30]:
parameters = {'tfidf__norm': ['l1', 'l2'],
              'clf__estimator__n_estimators': [100, 150],
              'clf__estimator__max_depth': [5, 15]

    }

cv = GridSearchCV(pipeline, param_grid = parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [31]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

In [32]:
display_results(y_test, y_pred, grid = cv)

Class - related
              precision    recall  f1-score   support

           0       0.94      0.97      0.95        31
           1       0.75      0.60      0.67         5

    accuracy                           0.92        36
   macro avg       0.84      0.78      0.81        36
weighted avg       0.91      0.92      0.91        36

Class - request
              precision    recall  f1-score   support

           0       0.69      1.00      0.81        24
           1       1.00      0.08      0.15        12

    accuracy                           0.69        36
   macro avg       0.84      0.54      0.48        36
weighted avg       0.79      0.69      0.59        36

Class - offer
              precision    recall  f1-score   support

           0       0.89      1.00      0.94        31
           1       1.00      0.20      0.33         5

    accuracy                           0.89        36
   macro avg       0.94      0.60      0.64        36
weighted avg       0.90     

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [39]:
pipeline_log = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultiOutputClassifier(LogisticRegression()))
    ])

In [40]:
pipeline_log.fit(X_train, y_train)

y_pred = pipeline_log.predict(X_test)

ValueError: ignored

In [ ]:
pipeline_log.get_params()

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.